In [40]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import json
import pandas as pd
import os
import numpy as np
import re
import jax.numpy as jnp
from jax import grad, jit, vmap, lax, nn, random
from optax import adam, apply_updates
os.environ['http_proxy'] = ''
os.environ['https_proxy'] = ''
os.environ['ftp_proxy'] = ''
os.environ['FTP_PROXY'] = ''
os.environ['ALL_PROXY'] = ''
os.environ['rsync_proxy'] = ''
os.environ['RSYNC_PROXY'] = ''
os.environ['HTTPS_PROXY'] = ''
os.environ['HTTP_PROXY'] = ''

In [6]:
"""Way to import Parcel Numbers from excel sheet"""
pnum_series = pd.read_excel("ParcelReport.xls")['Parcel number']
filt = pnum_series.apply(lambda row: len(str(row)) == 10)
pnum_series = pnum_series[filt]
print(pnum_series)

WARNING *** file size (85484) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
9      1723059037
10     1723059045
11     1723059046
12     1723059051
13     1723059075
          ...    
111    7226500100
112    7231501090
113    7231501965
114    7231502020
115    7231502025
Name: Parcel number, Length: 107, dtype: int64


In [7]:
with open("trial.json", "r") as fp:
    soups = json.load(fp)

In [8]:
trial_soup = BeautifulSoup(soups['1723059037'], 'html.parser')
# print(trial_soup.prettify())
# trial_soup.tr.table.div
trial_soup.findAll(text='R-8') == ['R-8']

True

In [9]:
data = {}
data['Parcel number'] = []
for i, pnum in pnum_series.iteritems():
    soup = BeautifulSoup(soups[str(pnum)])
    if trial_soup.findAll(text='R-8') != ['R-8']:
        continue
    data['Parcel number'].append(pnum)
    for tr in soup.tr.table.find_all('tr', {'class': re.compile('GridView*')})[:]:
        td = tr.find_all('td')
        if len(td) == 2:
            if td[0].text not in data:
                data[td[0].text] = []
            if td[1].text == '\xa0' or td[1].text.strip() == '':
                data[td[0].text].append(np.nan)
            else:
                data[td[0].text].append(td[1].text)

final_data = {}
for key in data.keys():
    final_data[key] = []
final_data['Sale Date'] = []
final_data['Sale Price'] = []
# https://blue.kingcounty.com/Assessor/eRealProperty/Detail.aspx?ParcelNbr=7224000390
i = 0
for _, pnum in pnum_series.iteritems():
    soup = BeautifulSoup(soups[str(pnum)])
    if trial_soup.findAll(text='R-8') != ['R-8']:
        continue
    i += 1
    final_data['Parcel number'].append(pnum)
    max = 0
    final_data['Sale Date'].append(np.nan)
    final_data['Sale Price'].append(np.nan)
    for tr in soup.tr.table.find_all('tr', {'class': re.compile('GridView*')})[:]:
        td = tr.find_all('td')
        if len(td) == 2:
            if len(final_data[td[0].text]) == i:
                continue
            if td[1].text == '\xa0' or td[1].text.strip() == '':
                final_data[td[0].text].append(np.nan)
            else:
                final_data[td[0].text].append(td[1].text)
        if len(td) == 8 and '$' in td[3].text and '/' in td[2].text:
            if (sale := float(td[3].text.strip("$").replace(',',''))) > max:
                max = sale
                final_data['Sale Date'][-1] = pd.to_datetime(td[2].text)
                final_data['Sale Price'][-1] = sale

    for key in final_data.keys():
        if len(final_data[key]) < i:
            final_data[key].append(np.nan)
        if len(final_data[key]) > i:
            raise RuntimeError(i, len(final_data[key]), pnum, key, "HERE")

In [10]:
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr'):
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            if i.xpath('.//td[6][contains(text(),"yes")]'):
                # Grabbing IP and corresponding PORT
                proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
                proxies.add(proxy)
    return proxies

In [11]:
new_df = pd.DataFrame.from_dict(final_data)

In [12]:
for key in final_data.keys():
    print('\n', key, len(final_data[key]))
    if len(final_data[key]) == 107:
        print(final_data[key])



 Parcel number 107
[1723059037, 1723059045, 1723059046, 1723059051, 1723059075, 1723059080, 1723059084, 1723059097, 1723059098, 1723059134, 1723059163, 7224000350, 7224000355, 7224000360, 7224000365, 7224000370, 7224000375, 7224000385, 7224000390, 7224000395, 7224000400, 7224000405, 7224000410, 7224000415, 7224000420, 7225000140, 7225000145, 7225000150, 7225000155, 7225000160, 7225000165, 7225000245, 7225000250, 7225000255, 7225000260, 7225000265, 7225000270, 7225000275, 7225000285, 7225000295, 7225000300, 7225000305, 7225000310, 7225000315, 7225000325, 7225000330, 7225000335, 7225000337, 7225000340, 7225000355, 7225000360, 7225000365, 7225000370, 7225000375, 7225000380, 7225000385, 7225000390, 7225000395, 7225000400, 7225000405, 7225000430, 7225000435, 7225000440, 7225000445, 7225000455, 7225000460, 7225500280, 7225500281, 7225500282, 7225500295, 7225500300, 7226000045, 7226000095, 7226000100, 7226000105, 7226000110, 7226000115, 7226000117, 7226000120, 7226000121, 7226000125, 7226000

In [13]:
pd.isnull(new_df.loc[45, 'Site Address'])

True

In [14]:
new_df[['Year Built', 'Grade', 'Sale Date', 'Sale Price']][new_df['Year Built'] == '1924']

,Year Built,Grade,Sale Date,Sale Price
3,1924,6 Low Average,1997-07-28,115000.0
14,1924,6 Low Average,2013-08-30,239500.0
62,1924,6 Low Average,2017-12-27,338000.0
77,1924,6 Low Average,2021-05-18,590000.0
80,1924,5 Fair,2020-04-22,530000.0
90,1924,5 Fair,2016-07-05,271000.0


In [15]:
new_df[new_df['Parcel number'] == 7226000117][['Year Built', 'Grade', 'Sale Date', 'Sale Price']]

,Year Built,Grade,Sale Date,Sale Price
77,1924,6 Low Average,2021-05-18,590000.0


In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.get('https://www.gps-coordinates.net/')
time.sleep(1)
address_element = driver.find_element(By.ID, 'address')
for index, address in zip(new_df.index, new_df['Site Address']):
    if pd.isnull(address):
        continue
    address_element.clear()
    address_element.send_keys(address+" Renton, WA")
    button = driver.find_element(By.CLASS_NAME, 'btn-primary')
    button.click()
    time.sleep(0.8)
    new_df.loc[index,'latitude'] = float(driver.find_element(By.ID, 'latitude').get_attribute('value'))
    new_df.loc[index,'longitude'] = float(driver.find_element(By.ID, 'longitude').get_attribute('value'))
time.sleep(1)
driver.quit()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
There is no [mac64] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/casieve/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


In [17]:
new_df[['Site Address', 'latitude', 'longitude']]

,Site Address,latitude,longitude
0,821 N 1ST ST 98055,47.484458,-122.205158
1,810 N RIVERSIDE DR 98055,47.484366,-122.205623
2,827 N 1ST ST 98055,47.484360,-122.204958
3,835 N 1ST ST 98055,47.484222,-122.204754
4,817 N 1ST ST 98055,47.484619,-122.205346
...,...,...,...
102,814 N 2ND ST 98057,47.485904,-122.205327
103,126 WELLS AVE S 98057,47.481708,-122.204944
104,116 WILLIAMS AVE S 98057,47.482074,-122.206324
105,121 WELLS AVE S 98057,47.481909,-122.205754


In [18]:
with open('redfin_scraper_data.db', 'wb') as file:
    new_df.to_pickle(file)

In [19]:
with open('redfin_scraper_data.db', 'rb') as file:
    df = pd.read_pickle(file)

In [21]:
df

,Parcel number,Parcel,Name,Site Address,Residential Area,Property Name,Jurisdiction,Levy Code,Property Type,Plat Block / Building Number,...,Laundry,Kitchens,# of Meals,Founder's Fee,Apt Conversion,Condo Land Type,Sale Date,Sale Price,latitude,longitude
0,1723059037,172305-9037,DOBSON WYMAN K+DOBSON VICKI,821 N 1ST ST 98055,\n025-003 (WC Appraisal District)\n,NaN,RENTON,2100,\nR\n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,47.484458,-122.205158
1,1723059045,172305-9045,WILLARD JACOB T+SARAH L CHR,810 N RIVERSIDE DR 98055,\n025-003 (WC Appraisal District)\n,NaN,RENTON,2100,\nR\n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-03,400000.0,47.484366,-122.205623
2,1723059046,172305-9046,827 N 1ST LLC,827 N 1ST ST 98055,\n025-003 (WC Appraisal District)\n,NaN,RENTON,2100,\nR\n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-10-25,368000.0,47.484360,-122.204958
3,1723059051,172305-9051,PETERSON MARY MICHELLE,835 N 1ST ST 98055,\n025-003 (WC Appraisal District)\n,NaN,RENTON,2100,\nR\n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1997-07-28,115000.0,47.484222,-122.204754
4,1723059075,172305-9075,SEVERANCE SHANNON R+RUTHIE,817 N 1ST ST 98055,\n025-003 (WC Appraisal District)\n,NaN,RENTON,2100,\nR\n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2011-02-24,298500.0,47.484619,-122.205346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,7226500100,722650-0100,WILBUR'S BLOC LLC,814 N 2ND ST 98057,\n025-003 (WC Appraisal District)\n,NaN,RENTON,2100,\nR\n,22,...,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-23,3800000.0,47.485904,-122.205327
103,7231501090,723150-1090,JKH PROPERTY MANAGEMENT LLC,126 WELLS AVE S 98057,NaN,AGP WEALTH ADVISORS,RENTON,2100,\nC\n,12,...,NaN,NaN,NaN,NaN,NaN,NaN,2004-01-16,400000.0,47.481708,-122.204944
104,7231501965,723150-1965,CBRI LLC,116 WILLIAMS AVE S 98057,\n025-003 (WC Appraisal District)\n,SINGLE FAMILY RES / TRANSFERED TO RESIDENTIAL ...,RENTON,2100,\nR\n,23,...,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-30,570000.0,47.482074,-122.206324
105,7231502020,723150-2020,WELLS LLC C/O KENNETH KING,121 WELLS AVE S 98057,NaN,Mixed Use Retail,RENTON,2100,\nC\n,23 &,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,47.481909,-122.205754


In [45]:
training = df[['Sale Date', 'Total Finished Area', 'Grade', 'Year Built', 'latitude', 'longitude', 'Sale Price']][df['Sale Price'] < 2000000].dropna()

In [124]:
def random_layer_params(m, n, key, scale=1e-2):
    """
    A helper function to randomly initialize weights and biases
    for a dense neural network layer
    """
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

def predict(params, in_descriptors):
    """
    Descriptors here are of shape (num_descriptors,)
    j is input_layer, i is output_layer
    j is number of descriptors
    (i, j) * (j,) -> (i,)
    b is shape (i,)
    (i,) + (i,)
    """
    activations = in_descriptors

    for w, b in params[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = nn.softplus(outputs)

    final_w, final_b = params[-1]
    logit = jnp.dot(final_w, activations) + final_b
    return logit

# Vectorization of predictions which batches over each A matrix row
batched_predict = vmap(predict, in_axes=(None, 0))

def loss(params, descriptors, targets):
    """
    Descriptors here are of shape (num_configs*num_atoms_per_config, num_descriptors)
    segment_sum contracts first axis from num_configs*num_atoms_per_config to num_configs
    Predictions are of shape (num_configs*num_atoms_per_config)
    Returns mean of the l2 norm of the differences between predictions and targets.
    """
    predictions = batched_predict(params, descriptors)
    return jnp.mean(jnp.square(predictions-targets))

# @partial(jit, static_argnames=['self', 'num_segments', 'optimizer'])
def update(params, x, y, optimizer, opt_state):
    grads = grad(loss)(params, x, y)
    updates, opt_state = optimizer.update(grads, opt_state)
    return apply_updates(params, updates), opt_state

In [178]:
num_epoch = 1000
layer_sizes = [6, 3, 1]
key = random.PRNGKey(0)
keys = random.split(key, len(layer_sizes))
params = [random_layer_params(m, n, k) for m, n, k in zip(layer_sizes[:-1],
                                                               layer_sizes[1:],
                                                               keys)]
learning_rate = 1.0e-3
optimizer = adam(learning_rate)
opt_state = optimizer.init(params)
targets = training['Sale Price']
descriptors = np.zeros((len(training), 6))
targets = np.zeros((len(training), 1))

In [56]:
(training['Sale Date'].max() - training['Sale Date'].min()).days
training

,Sale Date,Total Finished Area,Grade,Year Built,latitude,longitude,Sale Price
1,2020-02-03,"1,160",7 Average,1950,47.484366,-122.205623,400000.0
2,2016-10-25,"1,360",7 Average,1958,47.484360,-122.204958,368000.0
3,1997-07-28,"1,400",6 Low Average,1924,47.484222,-122.204754,115000.0
4,2011-02-24,"2,110",7 Average,1940,47.484619,-122.205346,298500.0
5,1985-11-05,"1,760",7 Average,1941,47.484758,-122.205575,75000.0
...,...,...,...,...,...,...,...
89,2020-10-28,"2,040",7 Average,1920,47.486249,-122.204745,620000.0
90,2016-07-05,800,5 Fair,1924,47.486355,-122.204739,271000.0
91,2018-06-12,"1,100",6 Low Average,1923,47.486457,-122.204765,417900.0
92,2021-09-15,"1,580",6 Low Average,1927,47.486602,-122.204929,631000.0


In [179]:
def s2f(some_val):
    try:
        return float(some_val)
    except ValueError:
        return float(some_val.replace(',', ''))

for i, timediff in enumerate(training['Sale Date'] - training['Sale Date'].min()):
    descriptors[i, 0] = timediff.days
    descriptors[i, 1:5] = [s2f(val) for val in training[['Total Finished Area', 'Year Built', 'latitude', 'longitude']].iloc[i]]
    descriptors[i, 5] = float(training['Grade'].iloc[i].split()[0])
    targets[i] = training['Sale Price'].iloc[i]

In [189]:
descriptors

array([[ 1.25080000e+04,  1.16000000e+03,  1.95000000e+03,
         4.74843664e+01, -1.22205623e+02,  7.00000000e+00],
       [ 1.13120000e+04,  1.36000000e+03,  1.95800000e+03,
         4.74843598e+01, -1.22204958e+02,  7.00000000e+00],
       [ 4.28300000e+03,  1.40000000e+03,  1.92400000e+03,
         4.74842220e+01, -1.22204754e+02,  6.00000000e+00],
       [ 9.24200000e+03,  2.11000000e+03,  1.94000000e+03,
         4.74846191e+01, -1.22205346e+02,  7.00000000e+00],
       [ 0.00000000e+00,  1.76000000e+03,  1.94100000e+03,
         4.74847583e+01, -1.22205575e+02,  7.00000000e+00],
       [ 4.31500000e+03,  1.05000000e+03,  1.94200000e+03,
         4.74849341e+01, -1.22205831e+02,  6.00000000e+00],
       [ 1.11230000e+04,  2.09000000e+03,  1.94100000e+03,
         4.75157786e+01, -1.22206139e+02,  7.00000000e+00],
       [ 7.39300000e+03,  1.21000000e+03,  1.92500000e+03,
         4.74841431e+01, -1.22205240e+02,  6.00000000e+00],
       [ 8.17900000e+03,  1.28000000e+03,  1.929

In [180]:
from time import time
fit_start = time()
in_mean = np.mean(descriptors)
in_std = np.std(descriptors)
inputs = (descriptors - in_mean) / in_std
out_mean = np.mean(targets)
out_std = np.std(targets)
targets = (targets - out_mean) / out_std
for epoch in range(num_epoch):
    # need to get take sub(A) which is of length num_configs*num_atoms_per_config
    batch_start = time()
    params, opt_state = update(params, inputs, targets, optimizer, opt_state)
    print(epoch, time()-batch_start, loss(params, inputs, targets))

0 0.017566919326782227 1.0000752
1 0.010465145111083984 1.0000196
2 0.01221919059753418 0.9999844
3 0.012908935546875 0.9999657
4 0.0205380916595459 0.99995726
5 0.012676000595092773 0.99995196
6 0.013615846633911133 0.99994445
7 0.013457059860229492 0.99993163
8 0.015713930130004883 0.9999124
9 0.023471832275390625 0.999887
10 0.01619100570678711 0.9998559
11 0.015687227249145508 0.9998198
12 0.0115509033203125 0.9997796
13 0.012271881103515625 0.99973536
14 0.0094451904296875 0.99968714
15 0.009506940841674805 0.99963474
16 0.013087987899780273 0.99957764
17 0.020611047744750977 0.99951565
18 0.012485265731811523 0.99944794
19 0.013039112091064453 0.9993742
20 0.02849411964416504 0.99929374
21 0.024505138397216797 0.99920624
22 0.014081954956054688 0.9991114
23 0.011809825897216797 0.9990088
24 0.012103796005249023 0.9988979
25 0.010688066482543945 0.99877864
26 0.010671854019165039 0.99865067
27 0.013445138931274414 0.9985137
28 0.013103008270263672 0.9983673
29 0.026875019073486328

In [192]:
for epoch in range(1000):
    # need to get take sub(A) which is of length num_configs*num_atoms_per_config
    batch_start = time()
    params, opt_state = update(params, inputs, targets, optimizer, opt_state)
    print(epoch, time()-batch_start, loss(params, inputs, targets))

0 0.035382747650146484 0.34078202
1 0.039292097091674805 0.34074044
2 0.032576799392700195 0.34069896
3 0.05029582977294922 0.34065735
4 0.03980112075805664 0.3406158
5 0.03330588340759277 0.34057423
6 0.030194997787475586 0.34053272
7 0.02547478675842285 0.34049118
8 0.038208961486816406 0.34044957
9 0.032562971115112305 0.340408
10 0.04421591758728027 0.34036645
11 0.03586411476135254 0.34032494
12 0.03911089897155762 0.34028336
13 0.04480600357055664 0.3402418
14 0.030755043029785156 0.34020025
15 0.03621220588684082 0.34015867
16 0.029349088668823242 0.34011716
17 0.028588056564331055 0.3400756
18 0.03251194953918457 0.34003407
19 0.044112205505371094 0.3399925
20 0.03905010223388672 0.33995095
21 0.03805112838745117 0.3399094
22 0.025561094284057617 0.33986786
23 0.028729915618896484 0.33982632
24 0.04387164115905762 0.33978474
25 0.04628133773803711 0.33974323
26 0.04359602928161621 0.33970165
27 0.02843499183654785 0.33966014
28 0.02848529815673828 0.3396186
29 0.030740976333618

In [206]:
our_house_in = np.zeros((1, 6))
our_house_in[0, :] = descriptors[66]
our_house_in[0, 0] += 360
our_house_in[0, 1] += 360
our_house_out = np.zeros((1, 1))
our_house_out[0, 0] = targets[66]
batched_predict(params, (our_house_in - in_mean) / in_std) * out_std + out_mean

DeviceArray([[994868.25]], dtype=float32)